<a href="https://colab.research.google.com/github/OsyaginVictor/A-project-for-Wikishop/blob/main/A_project_for_Wikishop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
# загрузим нужные библиотеки
import os
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

from nltk import word_tokenize, sent_tokenize, FreqDist
from collections import defaultdict, Counter
import string
import requests
from sklearn.feature_extraction.text import CountVectorizer

from tqdm.notebook import tqdm
tqdm.pandas()

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

In [ ]:
print('Данные')
display(df.head(8))
print('\n')
print('Размер датасета:', df.shape)
print('\n')
print('Информация\n')
print(df.info())
print('\n')
print('Описание\n')
print(df.describe())

Данные


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0




Размер датасета: (159292, 2)


Информация

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
None


Описание

               toxic
count  159292.000000
mean        0.101612
std         0.302139
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000


Напишем лемматизатор чтобы привести разные формы одного слова к его базовой форме, что облегчает анализ текста и извлечение смысла из него.

In [ ]:
lemmatizer = WordNetLemmatizer()

Далее, напишем функцию get_wordnet, которая будет использована для определения части речи (POS - part of speech) слова с помощью библиотеки NLTK (Natural Language Toolkit) и WordNet.

Она будет принимать на вход слово word и использует модуль pos_tag из NLTK для определения части речи этого слова. Затем, на основе полученной части речи, функция будет возвращать соответствующий символьный код части речи из WordNet (например, "J" для прилагательного, "N" для существительного, "V" для глагола, "R" для наречия).

Если часть речи слова не соответствует ни одному из указанных кодов, функция по умолчанию возвращает код для существительного (wordnet.NOUN).

In [ ]:
def get_wordnet(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

Теперь напишем функцию, которая будет выполнять очистку текста от лишних символов, лемматизацию слов и возвращать текст с лемматизированными словами.

In [ ]:
def clear(text):
    reg = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = reg.split()
    lemm = []

    for i in range(len(clear)):
      lemm.append(lemmatizer.lemmatize(clear[i], get_wordnet(clear[i])))
    return " ".join(lemm)

Так же, применим функцию к столбцу "text" с целью создания нового столбца, в котором будет содержаться лемматизированный текст.

In [ ]:
df['text'] = df['text'].str.lower()

In [ ]:
df['lemmatized_text'] = df['text'].progress_apply(clear)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [ ]:
df.head()

,text,toxic,lemmatized_text
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,d'aww! he matches this background colour i'm s...,0,d aww he match this background colour i m seem...
2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not try to edit war it s ju...
3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"you, sir, are my hero. any chance you remember...",0,you sir be my hero any chance you remember wha...


Выделим токсичные комментарии в отдельную переменную

In [ ]:
toxic = df[df['toxic'] ==1]

In [ ]:
toxic.head()

,text,toxic,lemmatized_text
6,cocksucker before you piss around on my work,1,cocksucker before you piss around on my work
12,hey... what is it..\n@ | talk .\nwhat is it......,1,hey what be it talk what be it an exclusive gr...
16,"bye! \n\ndon't look, come or think of comming ...",1,bye don t look come or think of comming back t...
42,you are gay or antisemmitian? \n\narchangel wh...,1,you be gay or antisemmitian archangel white ti...
43,"fuck your filthy mother in the ass, dry!",1,fuck your filthy mother in the as dry


Создадим переменную corpus,которая будет содержать подготовленный текстовый корпус, который будет использоваться для создания мешка слов.

In [ ]:
corpus = list(toxic['lemmatized_text'])

Далее:
1. Сформируем множество стоп-слов для английского языка с использованием библиотеки NLTK.
2. Создадим экземпляр класса CountVectorizer, который используется для преобразования корпуса текстов в мешок слов.
3. Устанавливим максимальное количество признаков в 25 наиболее часто встречающихся слов.
4. Передадим множество стоп-слов в экземпляр CountVectorizer для исключения их из мешка слов.
5. Выполним преобразование корпуса текстов в мешок слов (bag of words) с помощью функции fit_transform(), которая выделяет уникальные слова из корпуса и подсчитывает количество их вхождений в каждом тексте корпуса.
6. Вызовем словарь методом get_feature_names()

In [ ]:
stop_words = set(nltk_stopwords.words('english'))
count_vect = CountVectorizer(max_features=25, stop_words=stop_words)
bag_of_words = count_vect.fit_transform(corpus)

In [ ]:
count_vect.get_feature_names()

['article',
 'bitch',
 'block',
 'cunt',
 'die',
 'faggot',
 'fat',
 'fuck',
 'gay',
 'get',
 'go',
 'hate',
 'know',
 'like',
 'make',
 'moron',
 'nigger',
 'page',
 'people',
 'say',
 'shit',
 'suck',
 'talk',
 'user',
 'wikipedia']

Теперь выделим переменную без токсичных комментариев

In [ ]:
nontoxic_lem = df[df['toxic'] == 0]

In [ ]:
nontoxic_lem.head()

,text,toxic,lemmatized_text
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,d'aww! he matches this background colour i'm s...,0,d aww he match this background colour i m seem...
2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not try to edit war it s ju...
3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"you, sir, are my hero. any chance you remember...",0,you sir be my hero any chance you remember wha...


Теперь, проделаем такую же операцию с нетоксичными комментариями

In [ ]:
corpus = list(nontoxic_lem['lemmatized_text'])
stop_words = set(nltk_stopwords.words('english'))
count_vect = CountVectorizer(max_features=25, stop_words=stop_words)
bag_of_words = count_vect.fit_transform(corpus)

In [ ]:
count_vect.get_feature_names()

['add',
 'also',
 'article',
 'edit',
 'get',
 'go',
 'know',
 'like',
 'make',
 'may',
 'one',
 'page',
 'people',
 'please',
 'say',
 'see',
 'source',
 'talk',
 'think',
 'time',
 'use',
 'user',
 'well',
 'wikipedia',
 'would']

#### Подвывод

Мы удалили лишний столбец, провели очистку данных, отметили части речи и привели слова к их начальным формам (лемматизировали). Добавили столбец с лемматизированным текстом.

Создали мешок слов для каждого класса и определили наиболее часто встречающиеся слова в каждом из них.

Дальше, выделим целевой признак и приступим к обучении.

## Обучение

Выделим целевой признак

In [ ]:
target = df['toxic']
features = df['lemmatized_text']

Разделим датасет на обучающую и тестовую выборки.

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size=0.5, random_state=12345, stratify=target_test)

Преобразуем набор текстовых документов в матрицу TF-IDF признаков, используя установленные стоп-слова для фильтрации ненужных слов.

In [ ]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Векторизуем данные с учетом стоп-слов. Применим метод fit к обычающей выборке, а затем метод transform к обучающему и тестовому набору данных.

In [ ]:
train_tf_idf = count_tf_idf.fit_transform(features_train)
valid_tf_idf = count_tf_idf.transform(features_valid)
test_tf_idf = count_tf_idf.transform(features_test)

### Приступим к обучении моделей

#### Логистическая регрессия

In [ ]:
for c in range(1, 10):
    model = LogisticRegression(class_weight='balanced',
                          random_state=12345,
                          max_iter=1000,
                          solver='lbfgs',
                               C=c)

    model.fit(train_tf_idf, target_train)
    predicted_valid = model.predict(valid_tf_idf)
    print ("Значение С:", c)
    print("F1_score:", f1_score(target_valid, predicted_valid))
    print('')
print()

Значение С: 1
F1_score: 0.7498640565524742

Значение С: 2
F1_score: 0.7590027700831026

Значение С: 3
F1_score: 0.7587363712608332

Значение С: 4
F1_score: 0.7594259988745077

Значение С: 5
F1_score: 0.7599943494843905

Значение С: 6
F1_score: 0.7625674524282875

Значение С: 7
F1_score: 0.7631278538812786

Значение С: 8
F1_score: 0.7632559472628261

Значение С: 9
F1_score: 0.762451557341754




#### Случайный лес

In [ ]:
for depth in range(3, 20, 3):
    model = RandomForestClassifier(random_state=12345,
                                   max_depth = depth,
                                   class_weight='balanced')
    model.fit(train_tf_idf, target_train)
    predicted_valid = model.predict(valid_tf_idf)
    print('Глубина:', depth)
    print("F1_score:", f1_score(target_valid, predicted_valid))
    print('')
print()

Глубина: 3
F1_score: 0.3275575599335391

Глубина: 6
F1_score: 0.3862810841017044

Глубина: 9
F1_score: 0.38968360228313714

Глубина: 12
F1_score: 0.38957011980267797

Глубина: 15
F1_score: 0.38963423245304335

Глубина: 18
F1_score: 0.4058849622948396




In [ ]:
model = LogisticRegression(class_weight='balanced',
                          random_state=12345,
                          max_iter=1000,
                          solver='lbfgs',
                          C=8)

features_train_lr = pd.concat([features_train, features_valid])
target_train_lr = pd.concat([target_train, target_valid])

features_train_tf_idf = count_tf_idf.fit_transform(features_train_lr)
features_test_tf_idf = count_tf_idf.transform(features_test)
model.fit(features_train_tf_idf, target_train_lr)
predicted_test = model.predict(features_test_tf_idf)
print("F1_score:", f1_score(target_test, predicted_test))

F1_score: 0.7514910536779323


## Выводы

Была проведена предобработка данных, включающая разметку частей речи и лемматизацию слов (приведение их к начальным формам). Был создан столбец с лемматизированным текстом.

После этого был построен мешок слов для каждого класса и были выявлены наиболее употребимые слова в каждом из них.

После обучения моделей с подбором гиперпараметров лучший результат показала модель логистической регрессии. Значение F1-меры на тестовой выборке составило 0,75.